In [1]:
from torch_geometric.loader import DataLoader, ImbalancedSampler
from torch_geometric.transforms import FaceToEdge, OneHotDegree
import torchvision.transforms as transforms

from mantra.simplicial import SimplicialDataset
from mantra.transforms import (
    TriangulationToFaceTransform,
    OrientableToClassTransform,
    DegreeTransform,
)
from validation.validate_homology import validate_betti_numbers

import torch

In [2]:
tr = transforms.Compose(
    [
        TriangulationToFaceTransform(),
        FaceToEdge(remove_faces=False),
        DegreeTransform(),
        OrientableToClassTransform(),
        OneHotDegree(max_degree=8,cat=False)
    ]
)

dataset = SimplicialDataset(root="./data", transform=tr)


print()
print(f"Dataset: {dataset}:")
print("====================")
print(f"Number of graphs: {len(dataset)}")
print(f"Number of features: {dataset.num_features}")
print(f"Number of classes: {dataset.num_classes}")

data = dataset[0]  # Get the first graph object.

print()
print(data)
print("=============================================================")

# Gather some statistics about the first graph.
print(f"Number of nodes: {len(data.x)}")
print(f"Number of edges: {data.num_edges}")
print(f"Average node degree: {data.num_edges / len(data.x):.2f}")
print(f"Has isolated nodes: {data.has_isolated_nodes()}")
print(f"Has self-loops: {data.has_self_loops()}")
print(f"Is undirected: {data.is_undirected()}")

print("=============================================================")
print(f"Number of orientable Manifolds: {sum(dataset.orientable)}")
print(
    f"Number of non-orientable Manifolds: {len(dataset) - sum(dataset.orientable)}"
)
print(
    f"Percentage: {sum(dataset.orientable) / len(dataset):.2f}, {(len(dataset) - sum(dataset.orientable)) / len(dataset):.2f}"
)


Dataset: SimplicialDataset(712):
Number of graphs: 712
Number of features: 9


c:\Users\Ernst\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_geometric\data\storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'name', 'face', 'orientable', 'torsion_coefficients', 'dimension', 'betti_numbers', 'genus', 'n_vertices'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


Number of classes: 2

Data(dimension=[1], n_vertices=[1], torsion_coefficients=[3], betti_numbers=[3], orientable=[1], genus=[1], name='S^2', face=[3, 4], edge_index=[2, 12], x=[4, 9], y=[1])
Number of nodes: 4
Number of edges: 12
Average node degree: 3.00
Has isolated nodes: False
Has self-loops: False
Is undirected: True
Number of orientable Manifolds: 193
Number of non-orientable Manifolds: 519
Percentage: 0.27, 0.73


In [7]:
torch.tensor([data.y for data in dataset])

c:\Users\Ernst\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_geometric\data\storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'name', 'face', 'orientable', 'torsion_coefficients', 'dimension', 'betti_numbers', 'genus', 'n_vertices'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


tensor([ True,  True,  True, False,  True,  True,  True, False,  True, False,
        False,  True,  True,  True,  True,  True,  True, False,  True, False,
        False, False,  True, False, False, False, False, False, False, False,
        False, False, False, False,  True,  True,  True,  True,  True, False,
        False, False,  True,  True,  True, False,  True,  True,  True,  True,
        False, False,  True, False,  True,  True,  True,  True,  True,  True,
         True, False,  True,  True,  True,  True, False, False, False, False,
         True,  True,  True, False, False, False, False, False, False, False,
         True, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False,  True,  True,
         True, False, False, False, False, False, False,  True, False, False,
         True,  True,  True, False, False,  True, False, False, False, False,
        False, False, False, False, False,  True,  True, False, 

In [8]:
dataset = dataset.shuffle()

train_dataset = dataset[:-150]
test_dataset = dataset[-150:]

print(f"Number of training graphs: {len(train_dataset)}")
print(f"Number of test graphs: {len(test_dataset)}")


Number of training graphs: 562
Number of test graphs: 150


In [20]:
train_loader = DataLoader(train_dataset,batch_size=10)#,sampler=ImbalancedSampler(train_dataset))
test_loader = DataLoader(test_dataset,batch_size=10)


for batch in train_loader:
    break

batch.y

c:\Users\Ernst\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_geometric\data\storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'betti_numbers', 'face', 'n_vertices', 'name', 'dimension', 'orientable', 'torsion_coefficients', 'genus'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


tensor([1, 1, 0, 0, 1, 1, 0, 0, 0, 1])

In [10]:
from torch.nn import Linear
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
from torch_scatter import segment_coo

class PermInvariant(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        # torch.manual_seed(12345)
        self.classification = nn.Sequential( 
            nn.Linear(9,hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels,hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels,2),
            nn.ReLU()
        )

    def forward(self, batch):
        x = self.classification(batch.x)
        # print(batch.x)
        # print(x)
        return segment_coo(x,batch.batch,reduce="sum")


model = PermInvariant(hidden_channels=64)


In [12]:
model = PermInvariant(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()


def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        out = model(data
        )  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.


def test(loader):
    model.eval()

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data)
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int(
            (pred == data.y).sum()
        )  # Check against ground-truth labels.
    return correct / len(
        loader.dataset
    )  # Derive ratio of correct predictions.


for epoch in range(1, 40):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(
        f"Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}"
    )

Epoch: 001, Train Acc: 0.7313, Test Acc: 0.7200
Epoch: 002, Train Acc: 0.7384, Test Acc: 0.7333
Epoch: 003, Train Acc: 0.7811, Test Acc: 0.7600
Epoch: 004, Train Acc: 0.8025, Test Acc: 0.7800
Epoch: 005, Train Acc: 0.7972, Test Acc: 0.7733
Epoch: 006, Train Acc: 0.7972, Test Acc: 0.7733
Epoch: 007, Train Acc: 0.7900, Test Acc: 0.7667
Epoch: 008, Train Acc: 0.7918, Test Acc: 0.7667
Epoch: 009, Train Acc: 0.7865, Test Acc: 0.7667
Epoch: 010, Train Acc: 0.7794, Test Acc: 0.7600
Epoch: 011, Train Acc: 0.7794, Test Acc: 0.7600
Epoch: 012, Train Acc: 0.7811, Test Acc: 0.7600
Epoch: 013, Train Acc: 0.7811, Test Acc: 0.7600
Epoch: 014, Train Acc: 0.7794, Test Acc: 0.7600
Epoch: 015, Train Acc: 0.7811, Test Acc: 0.7600
Epoch: 016, Train Acc: 0.7811, Test Acc: 0.7600
Epoch: 017, Train Acc: 0.7811, Test Acc: 0.7600
Epoch: 018, Train Acc: 0.7811, Test Acc: 0.7600
Epoch: 019, Train Acc: 0.7811, Test Acc: 0.7600
Epoch: 020, Train Acc: 0.7811, Test Acc: 0.7600
Epoch: 021, Train Acc: 0.7811, Test Acc:

In [7]:
model(batch)[:,0]

tensor([0.9635, 0.8106, 0.9215, 0.9906, 1.5490, 1.0627, 1.0325, 1.2307, 1.2846,
        2.0083], grad_fn=<SelectBackward0>)